In [195]:
import os
import signal
import numpy as np
import rospy
import subprocess 
import roslib; roslib.load_manifest("robot_comm")
import httplib
from robot_comm.srv import *
from wsg_50_common.srv import *
from std_srvs.srv  import Empty 
roslib.load_manifest("netft_rdt_driver")
from netft_rdt_driver.srv import Zero
from datetime import date
import errno
from rosgraph_msgs.msg import Log
from geometry_msgs.msg import TransformStamped
import tf.transformations as tfm
import sys, argparse

In [196]:
setCart = rospy.ServiceProxy('/robot1_SetCartesian', robot_SetCartesian)
getCart = rospy.ServiceProxy('/robot1_GetCartesian', robot_GetCartesian)
setJoint = rospy.ServiceProxy('/robot1_SetJoints', robot_SetJoints)
setSpeed = rospy.ServiceProxy('/robot1_SetSpeed', robot_SetSpeed)
homeGripper = rospy.ServiceProxy('/wsg_50_driver/homing', Empty)
closeGripper = rospy.ServiceProxy('/wsg_50_driver/grasp', Move)
openGripper = rospy.ServiceProxy('/wsg_50_driver/release', Move)
setGripperForce = rospy.ServiceProxy('/wsg_50_driver/set_force', Conf)
zeroSensorFingerFront = rospy.ServiceProxy('/netft_1/zero', Zero)
zeroSensorFingerBack = rospy.ServiceProxy('/netft_2/zero', Zero)
zeroSensorPusher = rospy.ServiceProxy('/netft_3/zero', Zero)
clearBuffer = rospy.ServiceProxy('/robot1_ClearBuffer',robot_ClearBuffer)
addBuffer = rospy.ServiceProxy('/robot1_AddBuffer',robot_AddBuffer)
execBuffer = rospy.ServiceProxy('/robot1_ExecuteBuffer',robot_ExecuteBuffer)
setTool = rospy.ServiceProxy('/robot1_SetTool', robot_SetTool)
setSpeed = rospy.ServiceProxy('/robot1_SetSpeed', robot_SetSpeed)
getIK = rospy.ServiceProxy('/robot1_GetIK', robot_GetIK)
setJoints = rospy.ServiceProxy('/robot1_SetJoints', robot_SetJoints)

In [197]:
getCart()

x: 117.97
y: 365.97
z: -57.35
q0: 0.0394
qx: -0.0749
qy: -0.9964
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [222]:
#Length from robot phalange to center of the gripper finger.
toolZ0 = 145
setTool(0, 0, toolZ0, 0, 0, 1, 0)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [223]:
getCart()

x: 118.1
y: 345.0
z: -14.1
q0: 1.0
qx: 0.0
qy: 0.0
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [256]:
setSpeed(10,5)
# Set the z height such that the finger patch upper point aligns (1.5 mm above) with 
# the top edge of the object when the object is resting on the platform.
pos0 = [120, 345, -77]
quat0 = [1, 0, 0, 0]
setCart(pos0[0], pos0[1], pos0[2], quat0[0],quat0[1],quat0[2],quat0[3])

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [258]:
closeGripper(5.0, 5.0)

error: 0

In [251]:
openGripper(20, 5)

error: 0

In [266]:
#Move to touch the other piece
setTool(0, 0, toolZ0, 0, 0, 1, 0)
dx = 42.8
setCart(pos0[0] - dx, pos0[1], pos0[2], quat0[0],quat0[1],quat0[2],quat0[3])

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [259]:
zeroSensorFingerFront()
zeroSensorFingerBack()

In [260]:
# Simple geometry computation of COR w.r.t. the contact frame. 
h1 = 6.21
w1 = 76.00
# piece with rubber total thickness.
#h2 = 8.00
h2 = 6.80
# rough height from the center of the finger to the bottom of the piece/ground.
w2 = 76.00 - 19.8/2 + 1.5
num_thetas = 10
tot_theta = np.pi/45
cor = np.zeros((num_thetas, 2), float)
for i in range(num_thetas):
    theta = tot_theta * i/(num_thetas + 0.0) 
    print theta
    l2 = np.sqrt(h2*h2 - np.square(w1*np.sin(theta)))
    l1 = w1 * np.cos(theta) + h1 * np.sin(theta)

    p1 = np.array([0, h1])
    p3 = np.array([l1, w1 * np.sin(theta)])

    cor[i,:] = p3 + l2/l1 * (p3 - p1)

0.0
0.00698131700798
0.013962634016
0.0209439510239
0.0279252680319
0.0349065850399
0.0418879020479
0.0488692190558
0.0558505360638
0.0628318530718


In [261]:
w1,w2,l1, l2, p1, p3, cor

(76.0,
 67.6,
 76.23996048682568,
 4.8443015385308099,
 array([ 0.  ,  6.21]),
 array([ 76.23996049,   4.77207948]),
 array([[  8.28000000e+01,  -5.55631579e-01],
        [  8.28207706e+01,   2.42424747e-02],
        [  8.27959935e+01,   6.06554916e-01],
        [  8.27244912e+01,   1.19047794e+00],
        [  8.26041744e+01,   1.77520224e+00],
        [  8.24318165e+01,   2.35991555e+00],
        [  8.22026572e+01,   2.94377856e+00],
        [  8.19097044e+01,   3.52589441e+00],
        [  8.15424464e+01,   4.10526422e+00],
        [  8.10842620e+01,   4.68071374e+00]]))

In [262]:
for i in range(num_thetas):
    setTool(-(cor[i,0] - (w1 + h2/2)), 0, toolZ0 + w2 - cor[i,1], 0,0,1,0)
    theta = tot_theta * (i + 1 )/(num_thetas + 0.0) 
    angle = np.arcsin(w1*np.sin(theta)/h2) 
    print theta,angle
    q = np.zeros((4,1), float)
    q[0] = np.cos(angle/2.0)
    # rotation about y. 
    q[2] = np.sin(angle/2.0)
    cur_cart = getCart()
    print cur_cart,q
    print getIK(cur_cart.x, cur_cart.y, cur_cart.z, q[0], q[1], q[2], q[3])
    setCart(cur_cart.x, cur_cart.y, cur_cart.z, q[0], q[1], q[2], q[3])
    if i == num_thetas -1:
        angle = angle + 5.0/180.0 * np.pi
        q[0] = np.cos(angle/2.0)
        # rotation about y. 
        q[2] = np.sin(angle/2.0)
        push_in_dist = 10
        cur_cart.x = cur_cart.x - push_in_dist
        print getIK(cur_cart.x, cur_cart.y, cur_cart.z, q[0], q[1], q[2], q[3])
        setCart(cur_cart.x, cur_cart.y, cur_cart.z, q[0], q[1], q[2], q[3])

0.00698131700798 0.0781052387383
x: 80.6
y: 344.99
z: -145.2
q0: 1.0
qx: 0.0
qy: 0.0
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK. [[ 0.99923754]
 [ 0.        ]
 [ 0.03904269]
 [ 0.        ]]
j1: 19.2
j2: 0.54
j3: -16.43
j4: -43.82
j5: 21.53
j6: 131.75
ret: 1
msg: ROBOT_CONTROLLER: OK.
0.013962634016 0.15668825906
x: 80.62
y: 345.0
z: -144.6
q0: 0.9998
qx: -0.0017
qy: 0.0173
qz: 0.0002
ret: 1
msg: ROBOT_CONTROLLER: OK. [[ 0.99693267]
 [ 0.        ]
 [ 0.07826401]
 [ 0.        ]]
j1: 14.57
j2: -0.35
j3: -15.54
j4: -19.7
j5: 16.82
j6: 108.91
ret: 1
msg: ROBOT_CONTROLLER: OK.
0.0209439510239 0.236254004907
x: 80.69
y: 345.0
z: -144.0
q0: 0.9984
qx: -0.0024
qy: 0.057
qz: 0.0004
ret: 1
msg: ROBOT_CONTROLLER: OK. [[ 0.99303111]
 [ 0.        ]
 [ 0.11785247]
 [ 0.        ]]
j1: 9.77
j2: -0.49
j3: -15.4
j4: 13.53
j5: 16.32
j6: 77.0
ret: 1
msg: ROBOT_CONTROLLER: OK.
0.0279252680319 0.317366197647
x: 80.71
y: 345.0
z: -143.4
q0: 0.9953
qx: -0.0028
qy: 0.0963
qz: 0.0003
ret: 1
msg: ROBOT_CONTROLLER: 

In [263]:
    openGripper(15, 5)

error: 0

In [264]:
cart = getCart()
cart

x: 71.0
y: 345.0
z: -139.6
q0: 0.882
qx: 0.0
qy: 0.4712
qz: 0.0
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [267]:
cart = getCart()
setCart(cart.x+50, cart.y, cart.z + 100, cart.q0, cart.qx, cart.qy, cart.qz)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [95]:
setSpeed(10,5)
setCart(cart.x , cart.y, cart.z, 0.99, 0, 0.10, 0)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [131]:
q = getIK(cart.x, cart.y, cart.z, cart.q0, cart.qx, cart.qy, cart.qz)
#setJoints(q.j1, q.j2, q.j3, q.j4, q.j5, q.j6)
setCart(cart.x, cart.y, cart.z, cart.q0, cart.qx, cart.qy, cart.qz)

ret: 1
msg: ROBOT_CONTROLLER: OK.

In [110]:
np.arcsin(8.0/76) / np.pi * 180

6.0423284190541038

In [119]:
q

j1: 24.92
j2: -9.33
j3: 28.2
j4: -124.84
j5: 30.89
j6: 219.05
ret: 1
msg: ROBOT_CONTROLLER: OK.

In [125]:
-(cor[0,0] - (w1 + h2/2))

-4.0

In [165]:
5.0/180 * np.pi

0.08726646259971647

In [174]:
np.arcsin(np.sin(np.pi/40) * w1 / h2) 

0.84107679597673357